In [1]:
!pip uninstall torch torchdata dgl -y
!pip install torch==1.12.0
!pip install dgl==0.9.0
!pip install torchdata==0.4.0

Found existing installation: torch 1.12.0
Uninstalling torch-1.12.0:
  Successfully uninstalled torch-1.12.0
Found existing installation: torchdata 0.4.0
Uninstalling torchdata-0.4.0:
  Successfully uninstalled torchdata-0.4.0
Found existing installation: dgl 0.9.0
Uninstalling dgl-0.9.0:
  Successfully uninstalled dgl-0.9.0
  Using cached torch-1.12.0-cp310-cp310-manylinux1_x86_64.whl (776.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 1.12.0 which is incompatible.
torchtext 0.18.0 requires torch>=2.3.0, but you have torch 1.12.0 which is incompatible.
torchvision 0.18.0+cu121 requires torch==2.3.0, but you have torch 1.12.0 which is incompatible.
  Using cached dgl-0.9.0-cp310-cp310-manylinux1_x86_64.whl (6.2 MB)
  Using cached torchdata-0.4.0-cp310-cp310-manylinux2014_x86_64.whl (4.4 MB)

In [2]:
import os
os.environ['DGLBACKEND'] = 'pytorch'

import torch
import dgl

In [50]:
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import numpy as np

In [3]:
 torch.manual_seed(0)

In [4]:
num_nodes = 20
in_feats = 8
hid_feats = 32
out_feats = 2

In [5]:
s = torch.randint(low=0, high=num_nodes, size=(60,))
d = torch.randint(low=0, high=num_nodes, size=(60,))

In [6]:
myGraph = dgl.graph((s,d))

In [7]:
node_features = torch.randn(num_nodes, in_feats)
myGraph.ndata['features'] = node_features
edge_weights = torch.randn(60)
myGraph.edata['weights'] = edge_weights
myGraph.edge_index=torch.stack([s, d], dim=0)

In [8]:
labels = torch.randint(0, 2, (num_nodes,))
myGraph.ndata['labels'] = labels

In [9]:
train_mask = torch.ones(num_nodes, dtype=torch.bool)
myGraph.ndata['train_mask'] = train_mask

In [26]:
myGraph.ndata['train_mask'].shape

torch.Size([20])

In [49]:
train_mask

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True])

In [10]:
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

In [51]:
val_mask

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False])

In [11]:
train_mask[:10] = True
val_mask[10:15] = True
test_mask[15:] = True
myGraph.ndata['val_mask'] = val_mask
myGraph.ndata['test_mask'] = test_mask

In [29]:
myGraph.ndata['val_mask']

tensor([False, False, False, False, False, False, False, False, False, False,
         True,  True,  True,  True,  True, False, False, False, False, False])

In [ ]:
myGraph


Graph(num_nodes=20, num_edges=60,
      ndata_schemes={}
      edata_schemes={})

In [12]:
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import GraphConv

In [13]:
class GCN(nn.Module):
  def __init__(self,in_feat,hid_feat,out_feat):
     super(GCN, self).__init__()
     self.conv1=GraphConv(in_feat,hid_feat)
     self.conv2=GraphConv(hid_feat,out_feat)
     self.dropout=nn.Dropout(0.2)

  def forward (self,g,d_rate=0.2):
        x = self.conv1(g,g.ndata['features'])
        x = F.relu(x)
        x = F.dropout(x, p=d_rate, training=self.training)
        x = self.conv2(g, x)
        return x


In [52]:
model = GCN(in_feats,hid_feats,out_feats)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=.02)
schedulerGCN = StepLR(optimizer, step_size=10, gamma=0.5)

In [15]:
num_epochs=100

In [57]:
best_val_lossGCN = float('inf')
patienceGCN = 10
min_deltaGCN = 0.01
counterGCN = 0

In [58]:
for e in range(num_epochs):
  model.train()
  logits = model(myGraph)
  loss = loss_fn(logits[myGraph.ndata['train_mask']], myGraph.ndata['labels'][myGraph.ndata['train_mask']])
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  with torch.no_grad():
    model.eval()
    val_logits = model(myGraph)
    val_loss = loss_fn(val_logits[myGraph.ndata['val_mask']], myGraph.ndata['labels'][myGraph.ndata['val_mask']])

  schedulerGCN.step()
  if val_loss < best_val_lossGCN - min_deltaGCN:
        best_val_lossGCN = val_loss
        counterGCN = 0
  else:
        counterGCN += 1

  if counterGCN >= patienceGCN:
        print("Early stopping triggered")
        break
  print("for epoch",e,"  train loss is ",loss.item(),"     val loss  ",val_loss.item())

for epoch 0   train loss is  0.3657657504081726      val loss   0.6731125712394714
for epoch 1   train loss is  0.36390408873558044      val loss   0.648643434047699
for epoch 2   train loss is  0.3582181930541992      val loss   0.6239619255065918
for epoch 3   train loss is  0.34318870306015015      val loss   0.5991392135620117
for epoch 4   train loss is  0.3498222231864929      val loss   0.5686957240104675
for epoch 5   train loss is  0.30927032232284546      val loss   0.5409228205680847
for epoch 6   train loss is  0.3153562843799591      val loss   0.5188938975334167
for epoch 7   train loss is  0.31714102625846863      val loss   0.5098923444747925
for epoch 8   train loss is  0.31768277287483215      val loss   0.5009978413581848
for epoch 9   train loss is  0.3000843822956085      val loss   0.4908103048801422
for epoch 10   train loss is  0.2776080071926117      val loss   0.4835396707057953
for epoch 11   train loss is  0.29135847091674805      val loss   0.47190046310424

In [59]:
with torch.no_grad():
  model.eval()
  test_logits = model(myGraph)
  test_loss = loss_fn(test_logits[myGraph.ndata['test_mask']], myGraph.ndata['labels'][myGraph.ndata['test_mask']])
  print(f"Test Loss = {test_loss.item()}")

  _, predicted = torch.max(test_logits[myGraph.ndata['test_mask']], dim=1)
  accuracy = (predicted == myGraph.ndata['labels'][myGraph.ndata['test_mask']]).float().mean()
  print(f"Test Accuracy = {accuracy.item() * 100:.2f}%")

Test Loss = 0.21487149596214294
Test Accuracy = 100.00%


**Now Try GIN**




In [60]:
import torch
import dgl
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import GINConv
from dgl.nn.pytorch import SumPooling


In [19]:

class GIN(nn.Module):
    def __init__(self, in_feat, hid_feat, out_feat):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            nn.Sequential(
                nn.Linear(in_feat, hid_feat),
                nn.ReLU(),
                nn.Linear(hid_feat, hid_feat),
                nn.ReLU(),
                nn.BatchNorm1d(hid_feat),
            ),
            'sum'
        )
        self.conv2 = GINConv(
            nn.Sequential(
                nn.Linear(hid_feat, out_feat),
                nn.ReLU(),
                nn.Linear(out_feat, out_feat),
                nn.ReLU(),
                nn.BatchNorm1d(out_feat),
            ),
            'sum'
        )
        self.dropout=nn.Dropout(0.2)
    def forward (self,g,d_rate=0.2):
        x = self.conv1(g, g.ndata['features'])
        x = F.relu(x)
        x = F.dropout(x, p=d_rate, training=self.training)
        x = self.conv2(g, x)
        return x


In [63]:
modelGIN = GIN(in_feats,hid_feats,out_feats)
loss_fn_GIN = nn.CrossEntropyLoss()
optimizer_GIN = torch.optim.Adam(modelGIN.parameters(),lr=.02)
schedulerGIN = StepLR(optimizer, step_size=10, gamma=0.5)

In [21]:
num_epochs=100

In [61]:
best_val_lossGIN = float('inf')
patienceGIN = 10
min_deltaGIN = 0.01
counterGIN = 0

In [64]:
for e in range(num_epochs):
  modelGIN.train()
  logits = modelGIN(myGraph)
  loss = loss_fn_GIN(logits[myGraph.ndata['train_mask']], myGraph.ndata['labels'][myGraph.ndata['train_mask']])
  optimizer_GIN.zero_grad()
  loss.backward()
  optimizer_GIN.step()
  with torch.no_grad():
    model.eval()
    val_logits = modelGIN(myGraph)
    val_loss = loss_fn_GIN(val_logits[myGraph.ndata['val_mask']], myGraph.ndata['labels'][myGraph.ndata['val_mask']])
  schedulerGIN.step()
  if val_loss < best_val_lossGIN - min_deltaGIN:
        best_val_lossGIN = val_loss
        counterGIN = 0
  else:
        counter += 1

  if counterGIN >= patienceGIN:
        print("Early stopping triggered")
        break
  print("for epoch",e,"  train loss is ",loss.item(),"     val loss  ",val_loss.item())

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


for epoch 0   train loss is  0.8811634182929993      val loss   0.39225947856903076
for epoch 1   train loss is  0.4974571168422699      val loss   0.2749970555305481
for epoch 2   train loss is  0.46577635407447815      val loss   0.3021317720413208
for epoch 3   train loss is  0.45242610573768616      val loss   0.30234381556510925
for epoch 4   train loss is  0.5015394687652588      val loss   0.4417112469673157
for epoch 5   train loss is  0.5150814056396484      val loss   0.33920222520828247
for epoch 6   train loss is  0.4061223864555359      val loss   0.4449973702430725
for epoch 7   train loss is  0.4126763343811035      val loss   0.3650593161582947
for epoch 8   train loss is  0.40389910340309143      val loss   0.45945191383361816
for epoch 9   train loss is  0.40012431144714355      val loss   0.3735613226890564
for epoch 10   train loss is  0.3903151750564575      val loss   0.4128277897834778
for epoch 11   train loss is  0.44320836663246155      val loss   0.3570365309

In [65]:
with torch.no_grad():
  model.eval()
  test_logits = modelGIN(myGraph)
  test_loss = loss_fn_GIN(test_logits[myGraph.ndata['test_mask']], myGraph.ndata['labels'][myGraph.ndata['test_mask']])
  print(f"Test Loss = {test_loss.item()}")

  _, predicted = torch.max(test_logits[myGraph.ndata['test_mask']], dim=1)
  accuracy = (predicted == myGraph.ndata['labels'][myGraph.ndata['test_mask']]).float().mean()
  print(f"Test Accuracy = {accuracy.item() * 100:.2f}%")

Test Loss = 0.2001727819442749
Test Accuracy = 100.00%


In [66]:
!pip install torch-geometric


In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv


In [36]:

class GAT(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, num_heads):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_dim, hidden_dim, heads=num_heads)
        self.conv2 = GATConv(hidden_dim * num_heads, out_dim, heads=1)
        self.dropout=nn.Dropout(0.2)
    def forward(self, data,d_rate=0.2):
        x, edge_index = data.ndata['features'], data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=d_rate, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [33]:
num_heads = 4

In [41]:
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import numpy as np

In [70]:
modelGAT = GAT(in_feats,hid_feats,out_feats,num_heads)
loss_fn_GAT = nn.CrossEntropyLoss()
optimizer_GAT = torch.optim.Adam(modelGAT.parameters(),lr=.003)
schedulerGAT = StepLR(optimizer, step_size=10, gamma=0.5)



In [30]:
num_epochs=100

In [69]:
best_val_lossGAT = float('inf')
patienceGAT = 10
min_deltaGAT = 0.01
counterGAT = 0

In [72]:
for e in range(num_epochs):
  modelGAT.train()
  logits = modelGAT(myGraph)
  loss = loss_fn_GAT(logits[myGraph.ndata['train_mask']], myGraph.ndata['labels'][myGraph.ndata['train_mask']])
  optimizer_GAT.zero_grad()
  loss.backward()
  optimizer_GAT.step()
  with torch.no_grad():
    model.eval()
    val_logits = modelGAT(myGraph)
    val_loss = loss_fn_GAT(val_logits[myGraph.ndata['val_mask']], myGraph.ndata['labels'][myGraph.ndata['val_mask']])
  schedulerGAT.step()
  if val_loss < best_val_lossGAT - min_deltaGAT:
        best_val_lossGAT = val_loss
        counterGAT = 0
  else:
        counterGAT += 1

  if counterGAT >= patienceGAT:
        print("Early stopping triggered")
        break
  print("for epoch",e,"  train loss is ",loss.item(),"     val loss  ",val_loss.item())

for epoch 0   train loss is  0.07195504754781723      val loss   0.016204100102186203
for epoch 1   train loss is  0.06451189517974854      val loss   0.028879040852189064
for epoch 2   train loss is  0.11055926978588104      val loss   0.04016738384962082
for epoch 3   train loss is  0.06824623048305511      val loss   0.04810553789138794
for epoch 4   train loss is  0.06089162081480026      val loss   0.06885048002004623
for epoch 5   train loss is  0.07339777052402496      val loss   0.07488447427749634
for epoch 6   train loss is  0.0945415124297142      val loss   0.06005709245800972
for epoch 7   train loss is  0.05407755821943283      val loss   0.04824599623680115
for epoch 8   train loss is  0.06264400482177734      val loss   0.04115303233265877
for epoch 9   train loss is  0.05885685607790947      val loss   0.027575263753533363
Early stopping triggered


In [73]:
with torch.no_grad():
  model.eval()
  test_logits = modelGAT(myGraph)
  test_loss = loss_fn_GAT(test_logits[myGraph.ndata['test_mask']], myGraph.ndata['labels'][myGraph.ndata['test_mask']])
  print(f"Test Loss = {test_loss.item()}")

  _, predicted = torch.max(test_logits[myGraph.ndata['test_mask']], dim=1)
  accuracy = (predicted == myGraph.ndata['labels'][myGraph.ndata['test_mask']]).float().mean()
  print(f"Test Accuracy = {accuracy.item() * 100:.2f}%")

Test Loss = 0.058197982609272
Test Accuracy = 100.00%
